**Naam** Sjoerd Beetsma<br>
**Klas:** V2A<br>
**Studentnummer:** 6189293

In [1]:
%matplotlib widget
import ipywidgets as widgets
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
%matplotlib inline

In [2]:
def mixing_problem_euler(h=1/60, steps=60*300):
    # initial values:
    x  = [0] # KG salt in tank B
    xv = [100] # Volume water in tank A
    y  = [20] # KG salt in tank B
    yv = [100] # Volume water in tank B
    t  = [0] # time

            # L/min from -> to
    o_a = 6 # outside -> tank A
    a_b = 3 # tank A  -> tank B
    a_o = 4 # tank A  -> outside
    b_a = 1 # tank B  -> tank A
    b_o = 2 # tank B  -> outside

    for step in range(steps):
        c_a = x[-1] / xv[-1] # current concentration salt in tank A
        c_b = y[-1] / yv[-1] # current concentration salt in tank B
        
        # Salt in A - Salt out A = rate of change of salt in kg/min
        roc_x = (( o_a * 0.2 ) + ( b_a * c_b )) - (( a_o * c_a ) + ( a_b * c_a ))
        # Salt in B - Salt out B = rate of change of salt in kg/min
        roc_y = ( a_b * c_a ) - (( b_o * c_b ) + ( b_a * c_b ))
        
        # Salt in a tank is current salt in tank + (salt going in - salt going out) * h
        x.append( x[-1] + roc_x * h ) # update salt tank A
        xv.append( xv[-1] + ( o_a + b_a - a_b - a_o) * h ) # update water volume tank A
        y.append( y[-1] + roc_y * h ) # update salt tank B
        yv.append( yv[-1] + ( a_b - b_a - b_o ) * h ) # update water volume tank B
        t.append( t[-1] + h ) # update time
        
    return t,x,xv,y,yv

In [3]:
def mixing_problem_heun(h=1/60, steps=60*300):
    # initial values:
    x  = [0] # KG salt in tank B
    xv = [100] # Volume water in tank A
    y  = [20] # KG salt in tank B
    yv = [100] # Volume water in tank B
    t  = [0] # Time

            # L/min from -> to
    o_a = 6 # outside -> tank A
    a_b = 3 # tank A  -> tank B
    a_o = 4 # tank A  -> outside
    b_a = 1 # tank B  -> tank A
    b_o = 2 # tank B  -> outside

    for step in range(steps):
        c_a = x[-1] / xv[-1] # current concentration salt in tank A
        c_b = y[-1] / yv[-1] # current concentration salt in tank B
        
        roc_x_euler = (( o_a * 0.2 ) + ( b_a * c_b )) - ((a_o + a_b) * c_a)
        x_euler = x[-1] + h * roc_x_euler
        
        roc_y_euler = ( a_b * c_a ) - ((b_o + b_a) * c_b)
        y_euler = y[-1] + h * roc_y_euler
        
        # calculate concentrations according to what the next step would be from Euler's method.
        c_a_euler = x_euler / xv[-1]
        c_b_euler = y_euler / yv[-1]
        # recalculate rate of change with the concentrations obtained from Euler
        # and take the average from both rates of change => rate of change according to Heun's method.
        x_heun = x[-1] + h * .5 * ((((o_a * 0.2) + (b_a * c_b_euler)) - ((a_o + a_b) * c_a_euler)) + roc_x_euler)
        y_heun = y[-1] + h * .5 * (((a_b * c_a_euler) - ((b_o + b_a) * c_b_euler)) + roc_y_euler)

        x.append( x_heun ) # update salt tank A
        xv.append( xv[-1] + h * ( o_a + b_a - a_b - a_o)) # update water volume tank A
        y.append( y_heun ) # update salt tank B
        yv.append( yv[-1] + h * ( a_b - b_a - b_o )) # update water volume tank B
        t.append( t[-1] + h ) # update time

    return t,x,xv,y,yv

In [4]:
def plotter(step_size, minutes,method):
    """
    Function to interactively plot the approximation of the Euler method and/or Heun method.
    Args:
        step_size: change in time per step, can be chosen from 1/60 to 15 with steps of 1/60.
        minutes: numbers to approximates: (minutes/step_size) = number of steps.
        method: method to approximate the solution with either Euler, Heun or Both.
    """ 
    if method == 'Forward Euler' or method == 'Both':
        # plot solution according to the forward euler method
        t,x,xv,y,yv = mixing_problem_euler(h=step_size, steps = int(minutes/step_size))
        plt.plot(t,np.array(x)/np.array(xv), label='tank A Euler', c='blue')
        plt.plot(t,np.array(y)/np.array(yv), label='tank B Euler', c='magenta')
    
    if method == 'Heun' or method == 'Both':
        # plot solution according to heun's method
        t,x,xv,y,yv = mixing_problem_heun(h=step_size, steps = int(minutes/step_size))
        plt.plot(t,np.array(x)/np.array(xv), label='tank A Heun', c='red')
        plt.plot(t,np.array(y)/np.array(yv), label='tank B Heun', c='black')
    
    # set title and axes labels
    plt.title(f'Change of salt over time with a stepsize of {round(step_size,2)} minutes')
    plt.xlabel('Time in minutes')
    plt.ylabel('Salt concentration kg/l')
    # set a legend at the right lower corner
    plt.legend(loc=0)
    plt.show()

interactive_plot = interactive(
                                plotter,step_size=widgets.FloatSlider(min=1/60, max=15, value=1/60, step=0.01, description='step size'),
                                minutes =widgets.IntSlider(min=1, max=360, value=240, step=1),
                                method = widgets.Dropdown(options=['Forward Euler', 'Heun', 'Both'],
                                                                 value='Both',description='Solver')
                              )

In [5]:
interactive_plot

interactive(children=(FloatSlider(value=0.016666666666666666, description='step size', max=15.0, min=0.0166666…